# Imports

In [1]:
!python --version

No pyvenv.cfg file


In [2]:
import gc
import os
import re
import tempfile
import time
from itertools import product
from pathlib import Path
from pprint import pprint
from typing import Any

import cv2
import json5
import numpy as np
import t3f
import tensorflow as tf
import tensorly as tl
import torch
import yt_dlp
from dotenv import load_dotenv
from memory_profiler import memory_usage
from tqdm import tqdm

%load_ext memory_profiler
load_dotenv()


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "E:\__git_projects\tensor-compression-methods\.venv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "E:\__git_projects\tensor-compression-methods\.venv\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "E:\__git_projects\tensor-compression-methods\.venv\Lib\site-packages\ipykernel\kernelapp.py", line 739, in

AttributeError: _ARRAY_API not found

SystemError: initialization of _pywrap_checkpoint_reader raised unreported exception

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
print(tf.config.list_physical_devices("GPU"))

# Some class and methods for logging of compression metrics of some methods

In [4]:
class MethodLogger:
    experiments_count = 1

    def __init__(self, method_name: str, method_input_tensor: np.ndarray, qualitative_metrics: dict[str, str], func, method_args: dict[str, Any]):
        self.name = method_name
        self.method_args = method_args
        self.qualitative_metrics = qualitative_metrics

        self.method_input_tensor = method_input_tensor

        self.func = func

        gpu_allocated_mem_usages, gpu_cached_mem_usages, ram_mem_usages, durations, result = self._run_method_with_tracking(func, **method_args)

        self.method_result = result

        self.quantitative_metrics = {
            "gpu_allocated_mem_usage": gpu_allocated_mem_usages,
            "gpu_cached_mem_usages": gpu_cached_mem_usages,
            "ram_mem_usage": ram_mem_usages,
            "duration": durations,
        }
        self.logs = {}

        gc.collect()
        # torch.cuda.empty_cache()

    def _run_method_with_tracking(self, func, *args, **kwargs):
        gpu_allocated_mem_usages, gpu_cached_mem_usages, ram_mem_usages, durations = [], [], [], []

        def wrapper():
            gc.collect()
            torch.cuda.empty_cache()
            torch.cuda.synchronize()

            torch.cuda.synchronize()
            gpu_allocated_memory_before = torch.cuda.memory_allocated()
            gpu_cached_memory_before = torch.cuda.memory_reserved()

            start_time = time.time()

            result = func(*args, **kwargs)

            torch.cuda.synchronize()
            end_time = time.time()

            gpu_allocated_memory_after = torch.cuda.memory_allocated()
            gpu_cached_memory_after = torch.cuda.memory_reserved()

            gpu_allocated_memory_used = gpu_allocated_memory_after - gpu_allocated_memory_before
            gpu_cached_memory_used = gpu_cached_memory_after - gpu_cached_memory_before
            ram_memory_used = memory_usage((func, args, kwargs))[-1]
            duration = end_time - start_time

            gc.collect()
            torch.cuda.empty_cache()
            torch.cuda.synchronize()

            return gpu_allocated_memory_used, gpu_cached_memory_used, ram_memory_used, duration, result

        for _ in tqdm(range(MethodLogger.experiments_count), desc="Эксперимент набора параметров"):
            gpu_allocated_memory_used, gpu_cached_memory_used, ram_memory_used, duration, result = wrapper()

            gpu_allocated_mem_usages.append(gpu_allocated_memory_used)
            gpu_cached_mem_usages.append(gpu_cached_memory_used)
            ram_mem_usages.append(ram_memory_used)
            durations.append(duration)

        resulted_gpu_allocated_mem_usages = MethodLogger._compute_stats(gpu_allocated_mem_usages)

        resulted_gpu_cached_mem_usages = MethodLogger._compute_stats(gpu_cached_mem_usages)

        resulted_gpu_allocated_mem_usages["mean"] /= 1024**2
        resulted_gpu_allocated_mem_usages["min"] /= 1024**2
        resulted_gpu_allocated_mem_usages["max"] /= 1024**2

        resulted_gpu_cached_mem_usages["mean"] /= 1024**2
        resulted_gpu_cached_mem_usages["min"] /= 1024**2
        resulted_gpu_cached_mem_usages["max"] /= 1024**2

        resulted_ram_mem_usages = MethodLogger._compute_stats(ram_mem_usages)
        resulted_durations = MethodLogger._compute_stats(durations)

        return resulted_gpu_allocated_mem_usages, resulted_gpu_cached_mem_usages, resulted_ram_mem_usages, resulted_durations, result

    @staticmethod
    def _compute_stats(data):
        data = np.array(data)

        if len(data) > 1:
            lower_bound, upper_bound = np.percentile(data, [5, 95])
            filtered_data = data[(data >= lower_bound) & (data <= upper_bound)]
        else:
            filtered_data = data

        return {
            "mean": np.mean(filtered_data),
            "min": np.min(filtered_data),
            "max": np.max(filtered_data),
        }


method_logs_list = []

In [13]:
def save_logs_to_file(method_logs: MethodLogger, is_test: bool = False):
    log_entry = {
        "method_name": method_logs.name,
        "method_args": method_logs.method_args.copy(),
        "qualitative_metrics": method_logs.qualitative_metrics.copy(),
        "quantitative_metrics": method_logs.quantitative_metrics.copy(),
    }
    excluded_data = ["tensor", "input_tensor", "tens", "sites"]
    for arg in excluded_data:
        log_entry["method_args"].pop(arg, None)

    if is_test:
        pprint(log_entry)
    else:
        log_dir_path = Path("../.cache")
        log_dir_path.mkdir(parents=True, exist_ok=True)

        log_file_path = log_dir_path / "method_logs.json"

        if not log_file_path.exists() or log_file_path.stat().st_size == 0:
            with log_file_path.open("r+", encoding="utf-8") as f:
                json5.dump([log_entry], f, ensure_ascii=False, indent=4)
        else:
            with log_file_path.open("r+", encoding="utf-8") as f:
                logs = json5.load(f)

                existing_log = next(
                    (log for log in logs if log["method_name"] == log_entry["method_name"] and log["method_args"] == log_entry["method_args"]), None
                )

                if existing_log:
                    existing_log.update(log_entry)
                else:
                    logs.append(log_entry)

                f.seek(0)
                json5.dump(logs, f, ensure_ascii=False, indent=4)
                f.truncate()


def get_tensors_size(*args):
    total = 0
    for arg in args:
        partial = 1
        for d in arg.shape:
            partial *= d
        total += partial
    return total


def load_logs_from_file(log_file_path):
    log_file = Path(log_file_path)
    if log_file.exists():
        with log_file.open(encoding="utf-8") as f:
            logs = json5.load(f)
            # print(f"Загруженные логи:")
            # pprint(logs, indent=4, width=100)
            return logs
    else:
        print(f"Файл {log_file_path} не найден.")
        return None


def normalize_frame_tensorly_tensortrain(frame):
    frame = (frame - np.min(frame)) / (np.max(frame) - np.min(frame))
    frame = np.clip(frame * 255, 0, 255)
    return frame.astype(np.uint8)

# Download video and extract frames from it

## Some functions to do it

In [6]:
def download_progress_hook(d):
    if d["status"] == "downloading":
        print(f"Downloading: {d['_percent_str']} at {d['_speed_str']} ETA: {d['_eta_str']}")
    elif d["status"] == "finished":
        print("Download complete!")


def extract_video_id(video_url):
    video_id_match = re.search(r"(?:v=|\/)([0-9A-Za-z_-]{11}).*", video_url)
    if video_id_match:
        return video_id_match.group(1)
    error_message = "Не удалось извлечь ID видео из URL"
    raise ValueError(error_message)


def download_youtube_video(video_url, cache_dir=None, proxy_url=None):
    if cache_dir:
        Path(cache_dir).mkdir(parents=True, exist_ok=True)
        video_id = extract_video_id(video_url)
        cache_video_path = Path(cache_dir) / f"{video_id}.mp4"
    else:
        cache_video_path = tempfile.NamedTemporaryFile(delete=False, suffix=".mp4").name

    if Path(cache_video_path).exists():
        print(f"Видео уже загружено и закешировано: {cache_video_path}")
        return cache_video_path

    ydl_opts = {
        "format": "best",
        "outtmpl": str(cache_video_path),
        "progress_hooks": [download_progress_hook],
    }

    if proxy_url:
        ydl_opts["proxy"] = proxy_url

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([video_url])

    print(f"Видео загружено и сохранено: {cache_video_path}")
    return cache_video_path


def extract_frames(video_path):
    cap = cv2.VideoCapture(video_path)

    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    frames = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)

    cap.release()

    return np.array(frames), fps, (width, height)


def process_frames(frames):
    processed_frames = []

    for frame in frames:
        b_channel, g_channel, r_channel = cv2.split(frame)

        merged_frame = cv2.merge((b_channel, g_channel, r_channel))

        processed_frames.append(merged_frame)

    return np.array(processed_frames)


def show_frames_as_video(frames):
    for frame in frames:
        cv2.imshow("Downloaded Video", frame)
        if cv2.waitKey(25) & 0xFF == ord("q"):
            break
    cv2.destroyAllWindows()


def save_frames_as_video(name, frames, fps, frame_size):
    output_path = f"../.cache/output_videos/{name}.mp4"

    Path("../.cache/output_videos").mkdir(parents=True, exist_ok=True)

    width, height = frame_size
    size = (width, height)

    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, size)

    for frame in frames:
        out.write(frame)

    out.release()
    print(f"Видео сохранено как {output_path}")

## Some params

In [7]:
video_url = "https://www.youtube.com/watch?v=eSKe2Vx-rpY"
proxy_url = os.getenv("PROXY_URL")
cache_dir = "../.cache/youtube"

## Check how functions work

In [8]:
video_path = download_youtube_video(video_url, cache_dir=cache_dir, proxy_url=proxy_url)

Видео уже загружено и закешировано: ..\.cache\youtube\eSKe2Vx-rpY.mp4


In [9]:
video_frames, original_fps, frame_size = extract_frames(video_path)

In [10]:
video_frames.shape

(440, 360, 202, 3)

In [11]:
# processed_video_frames = process_frames(video_frames)

## Check original video

In [31]:
# show_frames_as_video(processed_video_frames)

# Implementations of Decompositions methods

Some packages which can decompose some dense type of tensors, from [this](https://arxiv.org/pdf/2103.13756) paper


Decomposition methods which used:
1. Canonical Polyadic Decomposition as PARAllel FACtors analysis (aka PARAFAC aka CPD aka CP)
2. Tucker Decomposition
3. Tensor Train
4. some variants of its (Other)

Tensor types:
1. Dense (D)
2. Sparse (S)
3. BlockSparse (BS)
4. Symmetric
5. Supersymmetric

Target system:
1. CPU (C)
2. GPU(G)
3. Distributed Memory (D)



| Method name                                                                             | Decomposition methods implemented | Tensor Type | Platform | Language         | Git | PyPI | Want to check | Checked     |
|-----------------------------------------------------------------------------------------|-----------------------------------|------------|----------|------------------|------|------|--------------|-------------|
| [AdaTM](https://github.com/hpcgarage/AdaTM)                                             | CP                                | S          | C        | C                | +    | ?    |              |             |
| [BTAS](https://github.com/ValeevGroup/BTAS)                                             | CP, Tucker                        | nan        | C        | C++              | +    | ?    |              |             |
| [CP-CALS](https://github.com/HPAC/CP-CALS)                                              | CP, Other                         | D          | C, G     | C++, Mat         | +    |      | +            |             |
| [CSTF](https://github.com/ZacBlanco/cstf)                                               | Other                             | S          | D        | Scala            | +    | ?    |              |             |
| [D-Tucker](https://datalab.snu.ac.kr/dtucker/resources/DTucker-v1.0.tar.gz)             | Tucker, Other                     | D          | C        | Matlab           |      | ?    |              |             |
| [DFacTo](http://www.joonheechoi.com/research.)                                          | CP                                | S          | C, D     | C++              |      | ?    |              |             |
| [EXATN](https://github.com/ORNL-QCI/exatn)                                              | TensorTrain                       | D          | C, D, G  | C++, Py          | +    |      | +            |             |
| [Genten](https://gitlab.com/tensors/genten)                                             | CP                                | D, S       | C, G     | C++              | +    |      | +            |             |
| GigaTensor                                                                              | CP                                | D          | C        | C++, Python      |      | ?    |              |             |
| [ITensor](https://github.com/ITensor/ITensor)                                           | TensorTrain                       | D, BS      | C, G     | C++, Julia       | +    |      | +            |             |
| [multiway](https://cran.r-project.org/web/packages/multiway/index.html)                 | CP, Tucker, Other                 | D          | C        | R                |      | ?    |              |             |
| [N-way toolbox](http://www.models.life.ku.dk/nwaytoolbox/download)                      | CP, Tucker, Other                 | D          | C        | Matlab           |      | ?    |              |             |
| [ParCube](https://www.cs.ucr.edu/~epapalex/src/parCube.zip)                             | CP                                | S          | C        | Matlab           |      | ?    |              |             |
| [ParTensor](https://github.com/neurocom/partensor-toolbox)                              | CP                                | D          | C, G     | C++              | +    |      | +            |             |
| [ParTI!](https://github.com/hpcgarage/ParTI)                                            | CP, Tucker                        | S          | C, G     | C, CUDA, Mat     | +    | ?    |              |             |
| [PLANC](https://github.com/ramkikannan/planc)                                           | CP                                | S          | C, D     | C++              | +    | ?    |              |             |
| [PLS toolbox](https://eigenvector.com/software/pls-toolbox/)                            | CP          , Tucker              | D          | C        | Matlab           |      | ?    |              |             |
| [Pytensor](https://code.google.com/archive/p/pytensor/source/default/source)            | Tucker                            | D, S       | C        | Python           |      | ?    |              |             |
| [rTensor](https://github.com/jamesyili/rTensor)                                         | CP, Tucker, Other                 | D          | C        | R                | +    |      | +            |             |
| [rTensor (randomized)](https://github.com/erichson/rTensor)                             | CP                                | D          | C        | Python           | +    |      | +       +    |             |
| [scikit-tensor](https://github.com/mnick/scikit-tensor)                                 | CP, Tucker, Other                 | D, S       | C        | Python           | +    | +    | +    +   +   | too old     |
| [Scikit-TT](https://github.com/PGelss/scikit_tt)                                        | TensorTrain                       | D          | C        | Python           | +    |      |     +   +    |             |
| [SPALS](https://github.com/dehuacheng/SpAls)                                            | CP                                | S          | C        | C++              | +    | ?    |              |             |
| [SPARTan](https://github.com/kperros/SPARTan)                                           | Other                             | S          | C        | Matlab           | +    | ?    |              |             |
| [SPLATT](https://github.com/ShadenSmith/splatt)                                         | CP                                | S          | C, D     | C, C++, Oct, Mat | +    | ?    |              |             |
| [SuSMoST](https://susmost.com/downloads.html)                                           | TensorTrain, Other                | D          | C        | Python           |      | ?    |              |             |
| [T3F](https://github.com/Bihaqo/t3f)                                                    | TensorTrain                       | D          | C, G     | Python           | +    | +    | +    + +     | in progress |
| [TDALAB](https://github.com/andrewssobral/TDALAB)                                       | CP                                | D, S       | C        | Python, Matlab   | +    |      | +         +  |             |
| [TeNPy](https://github.com/tenpy/tenpy)                                        | TensorTrain                       | D          | C        | Python           | +    | +    | +      + +   | in progress |
| [Tensor Fox](https://github.com/felipebottega/Tensor-Fox)                               | CP                                | D, S       | C        | Python, Matlab   | +    | +    | +    + +     | ?           |
| [Tensor package](http://www.gipsa-lab.fr/~pierre.comon/TensorPackage/tensorPackage.html) | CP                                | D          | C        | Matlab           |      | ?    |              |             |
| [Tensor Toolbox](https://gitlab.com/tensors/tensor_toolbox)                             | CP, Tucker, Other                 | D, S       | C        | Matlab           | +    |      | +            |             |
| [tensor_decomposition](https://github.com/cyclops-community/tensor_decomposition)       | CP, Tucker                        | D          | C, D     | Python           | +    |      | +        +   |             |
| [TensorBox](https://github.com/phananhhuy/TensorBox)                                    | CP, Tucker, Other                 | D, S       | C        | Matlab           | +    |      | +            |             |
| [TensorD](https://github.com/Large-Scale-Tensor-Decomposition/tensorD)                  | CP, Tucker                        | D          | C, G     | Python           | ?    | ?    |              |             |
| [TensorLab](https://www.tensorlab.net)                                                  | CP, Tucker, Other                 | D, S       | C        | Matlab           |      | ?    |              |             |
| [TensorLab+](https://www.tensorlabplus.net)                                             | CP, Other                         | D, S       | C        | Matlab           |      | ?    |              |             |
| [TensorLy](https://github.com/tensorly/tensorly)                                        | CP, Tucker, TensorTrain, Other    | D          | C, G     | Python           | +    | +    | +       + +  | in progress |
| [Three-Way](https://github.com/cran/ThreeWay)                                           | CP, Tucker                        | D          | C        | R                | +    |      | +            |             |
| [TNR](https://github.com/ycyuustc/matlab)                                               | Other                             | D          | C        | Matlab           | +    |      | +            |             |
| [TT-Toolbox](https://github.com/oseledets/TT-Toolbox)                                   | TensorTrain                       | D          | C, D, G  | Matlab, Python   | +    |      | +       +    |             |
| [xerus](https://git.hemio.de/xerus/xerus/)                                              | TensorTrain                       | D, S       | C        | C++              | +    |      | +            |             |

## TensorLy

In [32]:
# {‘numpy’, ‘mxnet’, ‘pytorch’, ‘tensorflow’, ‘cupy’}
# backend variants for tensorly
# tl.set_backend('pytorch')
# with tl.backend_context(‘pytorch’): ... pass

# video_frames_cuda = tl.tensor(video_frames.copy()).to(device)
# video_frames_cuda = tl.tensor(video_frames.copy())

### Params

In [33]:
video_frames

array([[[[ 40,  43,  56],
         [ 40,  43,  56],
         [ 40,  43,  56],
         ...,
         [ 52,  55,  66],
         [ 56,  56,  68],
         [ 53,  53,  65]],

        [[ 40,  43,  56],
         [ 40,  43,  56],
         [ 41,  44,  57],
         ...,
         [ 60,  63,  74],
         [ 60,  60,  72],
         [ 50,  50,  62]],

        [[ 40,  43,  56],
         [ 41,  44,  57],
         [ 40,  43,  56],
         ...,
         [ 71,  74,  85],
         [ 69,  69,  81],
         [ 47,  47,  59]],

        ...,

        [[125, 128, 133],
         [119, 122, 127],
         [127, 130, 135],
         ...,
         [128, 128, 135],
         [128, 129, 139],
         [132, 133, 143]],

        [[130, 133, 138],
         [121, 124, 129],
         [126, 129, 134],
         ...,
         [132, 132, 139],
         [125, 126, 136],
         [133, 134, 144]],

        [[132, 135, 140],
         [123, 126, 131],
         [125, 128, 133],
         ...,
         [135, 135, 142],
        

### Tucker (tl.decomposition.tucker)

#### Params

In [34]:
tl.SVD_FUNS

['truncated_svd', 'symeig_svd', 'randomized_svd']

In [35]:
rank_param = (video_frames.shape[0] // 2, video_frames.shape[1], video_frames.shape[2], video_frames.shape[3])

n_iter_max_param = 100

svd_params = ["truncated_svd", "symeig_svd", "randomized_svd"]

init_params = ["svd", "random"]

backend_params = ["pytorch", "numpy"]

random_state_param = 42

total_iterations = len(list(product(svd_params, init_params, backend_params)))

#### Implementation

In [36]:
log_file_path = "../.cache/method_logs.json"
logs = load_logs_from_file(log_file_path)

for backend, svd_func, init_method in tqdm(product(backend_params, svd_params, init_params), desc="Проверка набора параметров", total=total_iterations):
    method_name = f"TensorLy_Tucker_{backend}_{svd_func}_{init_method}"

    if logs:
        existing_log = next(
            (
                log
                for log in logs
                if log["method_name"] == method_name
                and log["method_args"].get("init") == init_method
                and log["method_args"].get("svd") == svd_func
                and log["qualitative_metrics"].get("TensorLy backend") == backend
            ),
            None,
        )
        if existing_log:
            print(f"Пропущена итерация: логи уже существуют для {method_name}")
            continue

    gc.collect()
    torch.cuda.empty_cache()

    try:
        with tl.backend_context(backend):
            if backend == "pytorch":
                tensor_param = tl.tensor(video_frames.copy()).to(device)
            elif backend == "numpy":
                tensor_param = tl.tensor(video_frames.copy())

            method_logs = MethodLogger(
                method_name=method_name,
                method_input_tensor=tensor_param,
                qualitative_metrics={
                    "Language": "Python",
                    "Library": "TensorLy",
                    "TensorLy backend": f"{backend}",
                    "Tensor type": "Dense",
                    "Platform": "CPU, GPU",
                    "Decomposition method": "Tucker",
                },
                method_args={
                    "tensor": tensor_param,
                    "rank": rank_param,
                    "n_iter_max": n_iter_max_param,
                    "init": init_method,
                    "svd": svd_func,
                    "random_state": random_state_param,
                },
                func=tl.decomposition.tucker,
            )

        if backend == "pytorch":
            tensor_param = tl.tensor(tensor_param.cpu())

        method_logs_list.append(method_logs)

        core, factors = method_logs.method_result
        factors_numpy = []
        if backend == "pytorch":
            core = tl.tensor(core.cpu())
            for index, factor in enumerate(factors):
                factors[index] = tl.tensor(factor.cpu())

        reconstruct_frames_from_tensorly_tt_factors = tl.tucker_tensor.tucker_to_tensor((core, factors))

        method_logs.quantitative_metrics["compression_ratio"] = 100.0 * get_tensors_size(core, *factors) / get_tensors_size(tensor_param)

        method_logs.quantitative_metrics["frobenius_error"] = (
            100.0 * tl.norm(reconstruct_frames_from_tensorly_tt_factors - tensor_param) / tl.norm(tensor_param)
        ).item()

        save_logs_to_file(method_logs=method_logs, is_test=False)

        reconstructed_frames = []

        reconstructed_frames = [normalize_frame_tensorly_tensortrain(frame) for frame in reconstruct_frames_from_tensorly_tt_factors]

        save_frames_as_video(name=method_logs.name, frames=reconstructed_frames, fps=original_fps, frame_size=frame_size)
    except (torch.cuda.OutOfMemoryError, MemoryError) as e:
        print(f"Пропущена итерация из-за недостатка памяти: {backend}, {svd_func}, {init_method}. Ошибка: {e!s}")
        gc.collect()
        torch.cuda.empty_cache()
        continue

### Tensor Train - MPS (tensorly.decomposition.tensor_train)

#### Params

In [37]:
rank_param = [1, 500, 302, 500, 1]

svd_params = ["truncated_svd", "symeig_svd", "randomized_svd"]

backend_params = ["pytorch", "numpy"]

total_iterations = len(list(product(backend_params, svd_params)))

#### Implementation

In [46]:
log_file_path = "../.cache/method_logs.json"
logs = load_logs_from_file(log_file_path)

for backend, svd_func in tqdm(product(backend_params, svd_params), desc="Проверка набора параметров", total=total_iterations):
    method_name = f"TensorLy_TensorTrain_{backend}_{svd_func}"

    if logs:
        existing_log = next(
            (
                log
                for log in logs
                if log["method_name"] == method_name
                and log["method_args"].get("svd") == svd_func
                and log["qualitative_metrics"].get("TensorLy backend") == backend
            ),
            None,
        )
        if existing_log:
            print(f"Пропущена итерация: логи уже существуют для {method_name}")
            continue

    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.synchronize()

    try:
        with tl.backend_context(backend):
            if backend == "pytorch":
                tensor_param = tl.tensor(video_frames.copy()).to(device)
            elif backend == "numpy":
                tensor_param = tl.tensor(video_frames.copy())

            method_logs = MethodLogger(
                method_name=method_name,
                method_input_tensor=tensor_param,
                qualitative_metrics={
                    "Language": "Python",
                    "Library": "TensorLy",
                    "TensorLy backend": f"{backend}",
                    "Tensor type": "Dense",
                    "Platform": "CPU, GPU",
                    "Decomposition method": "TensorTrain",
                },
                method_args={
                    "input_tensor": tensor_param,
                    "rank": rank_param,
                    "svd": svd_func,
                },
                func=tl.decomposition.tensor_train,
            )

        if backend == "pytorch":
            tensor_param = tl.tensor(tensor_param.cpu())

        method_logs_list.append(method_logs)

        tt_factors = method_logs.method_result
        factors_numpy = []
        if backend == "pytorch":
            for tt_factor in tt_factors:
                factors_numpy.append(tt_factor.cpu())
        elif backend == "numpy":
            factors_numpy = tt_factors

        reconstruct_frames_from_tensorly_tt_factors = tl.tt_to_tensor(factors_numpy)

        method_logs.quantitative_metrics["compression_ratio"] = 100.0 * get_tensors_size(*factors_numpy) / get_tensors_size(tensor_param)

        method_logs.quantitative_metrics["frobenius_error"] = (
            100.0 * tl.norm(reconstruct_frames_from_tensorly_tt_factors - tensor_param) / tl.norm(tensor_param)
        ).item()

        save_logs_to_file(method_logs=method_logs)

        reconstructed_frames = []

        for i in range(len(reconstruct_frames_from_tensorly_tt_factors)):
            reconstructed_frames.append(normalize_frame_tensorly_tensortrain(reconstruct_frames_from_tensorly_tt_factors[i]))

        save_frames_as_video(name=method_logs.name, frames=reconstructed_frames, fps=original_fps, frame_size=frame_size)

    except (torch.cuda.OutOfMemoryError, MemoryError) as e:
        print(f"Пропущена итерация из-за недостатка памяти: {backend}, {svd_func}. Ошибка: {e!s}")
        gc.collect()
        torch.cuda.empty_cache()
        torch.cuda.synchronize()
        continue

Загруженные логи:


Проверка набора параметров:   0%|          | 0/6 [00:00<?, ?it/s]

Пропущена итерация: логи уже существуют для TensorLy_TensorTrain_pytorch_truncated_svd



Проверка набора параметров:  33%|███▎      | 2/6 [00:03<00:06,  1.71s/it]

Пропущена итерация из-за недостатка памяти: pytorch, symeig_svd. Ошибка: CUDA out of memory. Tried to allocate 177.30 GiB. GPU 0 has a total capacity of 11.00 GiB of which 8.82 GiB is free. Of the allocated memory 740.47 MiB is allocated by PyTorch, and 365.53 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)
Пропущена итерация: логи уже существуют для TensorLy_TensorTrain_pytorch_randomized_svd


...

Видео сохранено как ../.cache/output_videos/TensorLy_TensorTrain_numpy_truncated_svd.mp4



Проверка набора параметров:  83%|████████▎ | 5/6 [07:32<01:33, 93.62s/it] 

Пропущена итерация из-за недостатка памяти: numpy, symeig_svd. Ошибка: Unable to allocate 44.3 GiB for an array with shape (218160, 218160) and data type uint8



Проверка набора параметров: 100%|██████████| 6/6 [1:15:22<00:00, 753.82s/it] 

Видео сохранено как ../.cache/output_videos/TensorLy_TensorTrain_numpy_randomized_svd.mp4


## T3F

### Params

In [52]:
tensor_param = video_frames.copy().astype(np.float32)

rank_param = [1, 500, 302, 500, 1]

### Implementation

In [68]:
log_file_path = "../.cache/method_logs.json"
logs = load_logs_from_file(log_file_path)

method_name = "T3F_TensorTrain"

if logs:
    existing_log = next((log for log in logs if log["method_name"] == method_name), None)
    if existing_log:
        error_message = f"Пропущена итерация: логи уже существуют для {method_name}"
        raise error_message

gc.collect()

method_logs = MethodLogger(
    method_name=method_name,
    method_input_tensor=tensor_param,
    qualitative_metrics={
        "Language": "Python",
        "Library": "T3F",
        "Tensor type": "Dense",
        "Platform": "CPU, GPU",
        "Decomposition method": "TensorTrain",
    },
    method_args={
        "tens": tensor_param,
        "max_tt_rank": rank_param,
    },
    func=t3f.to_tt_tensor,
)

method_logs_list.append(method_logs)

tt_factors = method_logs.method_result

reconstruct_frames_from_t3f_tt_factors = t3f.full(tt_factors)

method_logs.quantitative_metrics["compression_ratio"] = 100.0 * get_tensors_size(*tt_factors.tt_cores) / get_tensors_size(tensor_param)

method_logs.quantitative_metrics["frobenius_error"] = (
    100.0 * np.linalg.norm(reconstruct_frames_from_t3f_tt_factors - tensor_param) / np.linalg.norm(tensor_param)
)

save_logs_to_file(method_logs=method_logs)

reconstructed_frames = []

for i in range(len(reconstruct_frames_from_t3f_tt_factors)):
    reconstructed_frames.append(normalize_frame_tensorly_tensortrain(reconstruct_frames_from_t3f_tt_factors[i]))

save_frames_as_video(name=method_logs.name, frames=reconstructed_frames, fps=original_fps, frame_size=frame_size)

Эксперимент набора параметров: 100%|██████████| 5/5 [03:33<00:00, 42.71s/it]


Видео сохранено как ../.cache/output_videos/T3F_TensorTrain.mp4


## TeNPy

### Params

In [25]:
# tensor_param = video_frames.copy().astype(np.float32)
#
# # Размерность физического индекса
# d = tensor_param.shape[-1]  # в вашем случае это 3
#
# # Создаем объект LegCharge
# leg = LegCharge.from_trivial(d)
#
# # Создаем объекты Site для каждого физического индекса, кроме последнего
# sites = [Site(leg) for _ in range(tensor_param.ndim - 1)]
#
# rank_param = [1, 500, 302, 500, 1]

### Implementation

In [26]:
# mps = MPS.from_full(sites=sites, psi=tensor_param, normalize=False)
#
# reconstructed_tensor = mps.to_full_tensor()

AttributeError: 'numpy.ndarray' object has no attribute 'has_label'

In [16]:
# log_file_path = '../.cache/method_logs.json'
# logs = load_logs_from_file(log_file_path)
#
# method_name = f"TeNPy_TensorTrain"
#
# if logs:
#     existing_log = next(
#         (log for log in logs if log['method_name'] == method_name),
#         None
#     )
#     if existing_log:
#         error_message = f"Пропущена итерация: логи уже существуют для {method_name}"
#         raise error_message
#
# gc.collect()
# torch.cuda.empty_cache()
# torch.cuda.synchronize()
#
# method_logs = MethodLogger(
#     method_name=method_name,
#     method_input_tensor=tensor_param,
#     qualitative_metrics={
#         "Language": "Python",
#         "Library": "TeNPy",
#         "Tensor type": "Dense",
#         "Platform": "CPU",
#         "Decomposition method": "TensorTrain",
#     },
#     method_args={
#         "sites": tensor_param,
#         "psi": rank_param,
#         "normalize": False,
#     },
#     func=tenpy.networks.mps.MPS.from_full
# )
#
# method_logs_list.append(method_logs)
#
# tt_factors = method_logs.method_result
#
# reconstruct_frames_from_tenpy_tt_factors = tt_factors.to_full_tensor()

Эксперимент набора параметров:   0%|          | 0/1 [00:00<?, ?it/s]


ValueError: unknown type of a

In [ ]:
# method_logs.quantitative_metrics['compression_ratio'] = (100.0 * get_tensors_size(*tt_factors) / get_tensors_size(tensor_param))

# method_logs.quantitative_metrics['frobenius_error'] = (
#         100.0 * np.linalg.norm(reconstruct_frames_from_tenpy_tt_factors - tensor_param) / tl.norm(
#         tensor_param))
#
# save_logs_to_file(method_logs=method_logs, is_test=True)
#
# reconstructed_frames = []
#
# for i in range(len(reconstruct_frames_from_tenpy_tt_factors)):
#     reconstructed_frames.append(normalize_frame_tensorly_tensortrain(reconstruct_frames_from_tenpy_tt_factors[i]))
#
# save_frames_as_video(name=method_logs.name, frames=reconstructed_frames, fps=original_fps, frame_size=frame_size)